In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name

# Please change to your path!!!
ROOT="/home/quanghgx/spark-excel"

# Assembly or download spark-excel and its dependencies
jars = [
    "/home/quanghgx/spark-excel/target/scala-2.12/spark-excel_2.12-0.13.7+63-744368f8-SNAPSHOT.jar",
    "/home/quanghgx/jars/poi-ooxml-schemas-4.1.2.jar",
    "/home/quanghgx/jars/commons-collections4-4.4.jar",
    "/home/quanghgx/jars/xmlbeans-3.1.0.jar"
]

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", ",".join(jars)) \
    .getOrCreate()

In [2]:
df = spark.read.format("excel") \
   .option("header", True) \
   .option("inferSchema", True) \
   .load(f"{ROOT}/src/test/resources/spreadsheets/ca_dataset/2019/*/*.xlsx") \
   .withColumn("file_name", input_file_name())


In [3]:
df.show(10)

+---+-----+-----------+-----------------+----------------+--------------+--------------+--------------+--------------+--------------------+
|Day|Month|Customer ID|    Customer Name|Standard Package|Extra Option 1|Extra Option 2|Extra Option 3|         Staff|           file_name|
+---+-----+-----------+-----------------+----------------+--------------+--------------+--------------+--------------+--------------------+
|  1|   11|      CA767|   Vũ Phương Thảo|           14000|          null|          null|          null|   Teresa Teng|file:///home/quan...|
|  2|   11|      CA768|      Lê Thị Trâm|            null|          null|          2000|          null|Marilyn Monroe|file:///home/quan...|
|  2|   11|      CA769|         Lê Trung|            null|          null|          1200|          null|   Teresa Teng|file:///home/quan...|
|  3|   11|      CA770| Nguyễn Thảo Hiền|            null|          null|          1700|          null|   Teresa Teng|file:///home/quan...|
|  3|   11|      CA7

In [4]:
from pyspark.sql.functions import regexp_extract

df.withColumn("file_name", regexp_extract('file_name', '.*/(.*)\.xlsx$', 1)) \
   .show(5)

+---+-----+-----------+----------------+----------------+--------------+--------------+--------------+--------------+---------+
|Day|Month|Customer ID|   Customer Name|Standard Package|Extra Option 1|Extra Option 2|Extra Option 3|         Staff|file_name|
+---+-----+-----------+----------------+----------------+--------------+--------------+--------------+--------------+---------+
|  1|   11|      CA767|  Vũ Phương Thảo|           14000|          null|          null|          null|   Teresa Teng|    ca_11|
|  2|   11|      CA768|     Lê Thị Trâm|            null|          null|          2000|          null|Marilyn Monroe|    ca_11|
|  2|   11|      CA769|        Lê Trung|            null|          null|          1200|          null|   Teresa Teng|    ca_11|
|  3|   11|      CA770|Nguyễn Thảo Hiền|            null|          null|          1700|          null|   Teresa Teng|    ca_11|
|  3|   11|      CA771|  Nguyễn Thu Huệ|            null|          null|          1800|          null|  

In [5]:
df = spark.read.format("excel") \
   .option("header", True) \
   .option("inferSchema", True) \
   .load(f"{ROOT}/src/test/resources/spreadsheets/ca_dataset/2019")

In [6]:
df.printSchema()
df.groupBy("Quarter").count().show()

root
 |-- Day: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Standard Package: integer (nullable = true)
 |-- Extra Option 1: integer (nullable = true)
 |-- Extra Option 2: integer (nullable = true)
 |-- Extra Option 3: integer (nullable = true)
 |-- Staff: string (nullable = true)
 |-- Quarter: integer (nullable = true)

+-------+-----+
|Quarter|count|
+-------+-----+
|      1|    7|
|      3|  186|
|      4|  224|
|      2|  161|
+-------+-----+



In [7]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
schema = StructType([
    StructField("RowID", IntegerType(), True),
    StructField("1", StringType(), True),
    StructField("2", StringType(), True),
    StructField("3", StringType(), True)  
])
df = spark.read.format("excel") \
    .schema(schema) \
    .option("header", False) \
    .option("columnNameOfRowNumber", "RowID") \
    .load(f"{ROOT}/src/test/resources/spreadsheets/issue_285_bryce21.xlsx")
df.show()

+-----+---------+----+----+
|RowID|        1|   2|   3|
+-----+---------+----+----+
|    0|File info|null|null|
|    1|     Info|Info|Info|
|    3| Metadata|null|null|
|    5|     null|   1|   2|
|    6|        A|   1|   2|
|    7|        B|   5|   6|
|    8|        C|   9|  10|
|   11| Metadata|null|null|
|   13|     null|   1|   2|
|   14|        A|   1|   2|
|   15|        B|   4|   5|
|   16|        C|   7|   8|
+-----+---------+----+----+



In [8]:
df = spark.read.format("excel") \
    .schema(schema) \
    .option("header", False) \
    .option("keepUndefinedRows", True) \
    .option("columnNameOfRowNumber", "RowID") \
    .load(f"{ROOT}/src/test/resources/spreadsheets/issue_285_bryce21.xlsx")
df.show()

+-----+---------+----+----+
|RowID|        1|   2|   3|
+-----+---------+----+----+
|    0|File info|null|null|
|    1|     Info|Info|Info|
| null|     null|null|null|
|    3| Metadata|null|null|
| null|     null|null|null|
|    5|     null|   1|   2|
|    6|        A|   1|   2|
|    7|        B|   5|   6|
|    8|        C|   9|  10|
| null|     null|null|null|
| null|     null|null|null|
|   11| Metadata|null|null|
| null|     null|null|null|
|   13|     null|   1|   2|
|   14|        A|   1|   2|
|   15|        B|   4|   5|
|   16|        C|   7|   8|
+-----+---------+----+----+

